# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [76]:
# import libraries
import nltk
nltk.download(['punkt', 'wordnet', 'stopwords'])

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/philipleblanc/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/philipleblanc/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/philipleblanc/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [155]:
import numpy as np
from sqlalchemy import create_engine
import pandas as pd
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer # different from nltk.stem.wordnet ?
from nltk.corpus import stopwords
import pickle
import re
import sys

from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer

In [201]:
# load data from database

# df = pd.read_sql_table('messageCategories',engine)
engine = create_engine('sqlite:///data/DisasterResponse.db')
df = pd.read_sql ('SELECT * FROM messageCategories', engine)
df.shape[0]

26216

In [207]:
# listdf.loc[:,'related':].sum().values

array([20282,  4474,   118, 10860,  2084,  1313,   724,   471,   860,
           0,  1672,  2923,  2314,   405,   604,   298,   875,  1194,
        3446,  1705,  1201,  1333,   532,   159,   283,   120,   309,
        1151,  7297,  2155,  2443,   282,  2455,   530,  1376,  5075])

In [212]:
df = df[df.related != 2]

In [213]:
df.unique()

array([1, 0])

In [118]:
df = df.dropna()
X = df['message']
y = df.loc[:,'related':]

In [119]:
print(df.shape[0])
y.head()

10223


,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,child_alone,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [92]:
X.head()

0    Weather update - a cold front from Cuba that c...
1              Is the Hurricane over or is it not over
2                      Looking for someone but no name
3    UN reports Leogane 80-90 destroyed. Only Hospi...
4    says: west side of Haiti, rest of the country ...
Name: message, dtype: object

In [58]:
from sklearn.datasets import make_multilabel_classification
X, y = make_multilabel_classification(n_classes=3, random_state=0)
len(y[0])

3

In [61]:
len(X[-2:])

2

### 2. Write a tokenization function to process your text data

In [190]:
def tokenize(text):
    stop_words = stopwords.words("english")
    lemmatizer = WordNetLemmatizer()
    
    # normalize case and remove punctuation
    text = re.sub(r"[^a-zA-Z0-9]", " ", text.lower())
    
    tokens = word_tokenize(text)

    # lemmatize andremove stop words
    tokens = [lemmatizer.lemmatize(word) for word in tokens if word not in stop_words]

    return tokens

In [94]:
tokenize(X[0])

['weather', 'update', 'cold', 'front', 'cuba', 'could', 'pas', 'haiti']

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [95]:
pipeline = Pipeline([
        ('vect', CountVectorizer(tokenizer=tokenize)),
        ('tfidf', TfidfTransformer()),
        ('clf', MultiOutputClassifier(RandomForestClassifier()))
    ])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [96]:
X_train, X_test, y_train, y_test = train_test_split(X, y)
pipeline.fit(X_train, y_train)

Pipeline(steps=[('vect',
                 CountVectorizer(tokenizer=<function tokenize at 0x150ef5680>)),
                ('tfidf', TfidfTransformer()),
                ('clf',
                 MultiOutputClassifier(estimator=RandomForestClassifier()))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [127]:
y_pred = pipeline.predict(X_test)

In [106]:
# convert to a DataFrame
y_pred = pd.DataFrame(y_pred, columns=y_test.columns)

In [115]:
report = []
for col in y.columns:
    report.append(classification_report(y_test[col], y_pred[col]))
print(report[0])

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _wa

              precision    recall  f1-score   support

         0.0       0.42      0.12      0.18       853
         1.0       0.66      0.92      0.77      1668
         2.0       0.00      0.00      0.00        35

    accuracy                           0.64      2556
   macro avg       0.36      0.35      0.32      2556
weighted avg       0.57      0.64      0.56      2556



/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _wa

In [130]:
print(report[2])

              precision    recall  f1-score   support

         0.0       1.00      1.00      1.00      2554
         1.0       0.00      0.00      0.00         2

    accuracy                           1.00      2556
   macro avg       0.50      0.50      0.50      2556
weighted avg       1.00      1.00      1.00      2556



In [145]:
precision, recall = report[0][report[0].find('weight'):].split()[2:4]

In [151]:
col = '0'
print(col + '\n' + f'precision: {precision}' + ' ' + f'recall: {recall}')

0
precision: 0.57 recall: 0.64


In [148]:
def display_results0(y_test, y_pred):
    
    accuracy = (y_pred == y_test).mean()
    print(type(accuracy))

    print("Accuracy:\n", accuracy)
display_results0(y_test, y_pred)

<class 'pandas.core.series.Series'>
Accuracy:
 related                   0.639280
request                   0.615415
offer                     0.999218
aid_related               0.580986
medical_help              0.944836
medical_products          0.966354
search_and_rescue         0.977308
security                  0.984351
military                  0.994523
child_alone               1.000000
water                     0.918232
food                      0.846635
shelter                   0.879499
clothing                  0.991002
money                     0.988263
missing_people            0.990219
refugees                  0.983177
death                     0.975352
other_aid                 0.839984
infrastructure_related    0.967527
transport                 0.980438
buildings                 0.958138
electricity               0.990219
tools                     0.996870
hospitals                 0.996088
shops                     0.997261
aid_centers               0.992958
other_in

### 6. Improve your model
Use grid search to find better parameters. 

In [120]:
pipeline.get_params()

{'memory': None,
 'steps': [('vect',
   CountVectorizer(tokenizer=<function tokenize at 0x150ef5680>)),
  ('tfidf', TfidfTransformer()),
  ('clf', MultiOutputClassifier(estimator=RandomForestClassifier()))],
 'verbose': False,
 'vect': CountVectorizer(tokenizer=<function tokenize at 0x150ef5680>),
 'tfidf': TfidfTransformer(),
 'clf': MultiOutputClassifier(estimator=RandomForestClassifier()),
 'vect__analyzer': 'word',
 'vect__binary': False,
 'vect__decode_error': 'strict',
 'vect__dtype': numpy.int64,
 'vect__encoding': 'utf-8',
 'vect__input': 'content',
 'vect__lowercase': True,
 'vect__max_df': 1.0,
 'vect__max_features': None,
 'vect__min_df': 1,
 'vect__ngram_range': (1, 1),
 'vect__preprocessor': None,
 'vect__stop_words': None,
 'vect__strip_accents': None,
 'vect__token_pattern': '(?u)\\b\\w\\w+\\b',
 'vect__tokenizer': <function __main__.tokenize(text)>,
 'vect__vocabulary': None,
 'tfidf__norm': 'l2',
 'tfidf__smooth_idf': True,
 'tfidf__sublinear_tf': False,
 'tfidf__use_i

In [121]:
parameters = {  'clf__estimator__n_estimators': [10, 20],
                'clf__estimator__min_samples_split': [2, 5]
             }
cv = GridSearchCV(pipeline, param_grid=parameters)

In [122]:
cv.fit(X_train, y_train)
y_pred = cv.predict(X_test)

NameError: name 'model' is not defined

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [125]:
def display_results(cv, y_test, y_pred):
    
    accuracy = (y_pred == y_test).mean()
    

    print("Accuracy:\n", accuracy)
    print("\nBest Parameters:", cv.best_params_)
display_results(cv, y_test, y_pred)

Accuracy: related                   0.625978
request                   0.604851
offer                     0.999218
aid_related               0.580595
medical_help              0.942097
medical_products          0.965571
search_and_rescue         0.977308
security                  0.983959
military                  0.994523
child_alone               1.000000
water                     0.917058
food                      0.842332
shelter                   0.876761
clothing                  0.991002
money                     0.988263
missing_people            0.990219
refugees                  0.983568
death                     0.973396
other_aid                 0.834116
infrastructure_related    0.967136
transport                 0.980047
buildings                 0.951487
electricity               0.990219
tools                     0.996870
hospitals                 0.995305
shops                     0.997261
aid_centers               0.992958
other_infrastructure      0.980829
weather_re

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

### 9. Export your model as a pickle file

In [ ]:
filename = 'finalized_model.sav'
pickle.dump(model, open(filename, 'wb'))

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.

In [191]:
def load_data(database_filepath):
    # load data from database
    engine = create_engine(f'sqlite:///{database_filepath}')
    df = pd.read_sql_table('messageCategories',engine)
    df = df.dropna()
    X = df['message']
    y = df.loc[:,'related':]
    category_names = y.columns
    
    return X, y, category_names

def tokenize(text):
    stop_words = stopwords.words("english")
    lemmatizer = WordNetLemmatizer()
    
    # normalize case and remove punctuation
    text = re.sub(r"[^a-zA-Z0-9]", " ", text.lower())
    
    tokens = word_tokenize(text)

    # lemmatize andremove stop words
    tokens = [lemmatizer.lemmatize(word) for word in tokens if word not in stop_words]

    return tokens


def build_model():
    pipeline = Pipeline([
        ('vect', CountVectorizer(tokenizer=tokenize)),
        ('tfidf', TfidfTransformer()),
        ('clf', MultiOutputClassifier(RandomForestClassifier()))
    ])
    return pipeline


def evaluate_model(model, X_test, Y_test, category_names):
    y_pred = model.predict(X_test)
    
    # convert to a DataFrame
    y_pred = pd.DataFrame(y_pred, columns=category_names)
    
    # iterate through the columns and calling sklearn's classification_report on each
    # report f1 score, precision and recall for each output category of the dataset
    for col in Y_test.columns:
        report = classification_report(Y_test[col], y_pred[col])
        precision, recall = report[report.find('weight'):].split()[2:4]
        print(col + ':\n' + f'precision: {precision}' + ' ' + f'recall: {recall}')
        
    Y_test = Y_test.reset_index(drop=True)  # indexes need to be identical in y_pred and Y_test
    
    accuracy = (y_pred == Y_test).mean()
    print("Accuracy:\n", accuracy)

def save_model(model, model_filepath):
    pickle.dump(model, open(model_filepath, 'wb'))






In [192]:
database_filepath, model_filepath = 'DisasterMessages.db', 'classifier.pkl'
print('Loading data...\n    DATABASE: {}'.format(database_filepath))
X, Y, category_names = load_data(database_filepath)
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2)

print('Building model...')
model = build_model()

print('Training model...')
model.fit(X_train, Y_train)

print('Evaluating model...')
evaluate_model(model, X_test, Y_test, category_names)

print('Saving model...\n    MODEL: {}'.format(model_filepath))
save_model(model, model_filepath)

print('Trained model saved!')

Loading data...
    DATABASE: DisasterMessages.db
Building model...
Training model...
Evaluating model...


/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _wa

related:
precision: 0.57 recall: 0.63
request:
precision: 0.59 recall: 0.64
offer:
precision: 1.00 recall: 1.00
aid_related:
precision: 0.57 recall: 0.60
medical_help:
precision: 0.89 recall: 0.94
medical_products:
precision: 0.94 recall: 0.97
search_and_rescue:
precision: 0.94 recall: 0.97
security:
precision: 0.98 recall: 0.99
military:
precision: 1.00 recall: 1.00
child_alone:
precision: 1.00 recall: 1.00
water:
precision: 0.87 recall: 0.92
food:
precision: 0.75 recall: 0.84
shelter:
precision: 0.79 recall: 0.88
clothing:
precision: 0.98 recall: 0.99
money:
precision: 0.98 recall: 0.99
missing_people:
precision: 0.98 recall: 0.99
refugees:
precision: 0.96 recall: 0.98
death:
precision: 0.94 recall: 0.97
other_aid:
precision: 0.77 recall: 0.86
infrastructure_related:
precision: 0.95 recall: 0.98
transport:
precision: 0.96 recall: 0.98
buildings:
precision: 0.92 recall: 0.96
electricity:
precision: 0.99 recall: 1.00
tools:
precision: 1.00 recall: 1.00
hospitals:
precision: 0.99 recall

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _wa

Accuracy:
 related                   0.634230
request                   0.637653
offer                     0.999511
aid_related               0.600978
medical_help              0.940831
medical_products          0.967237
search_and_rescue         0.970660
security                  0.989731
military                  0.997555
child_alone               1.000000
water                     0.923227
food                      0.844499
shelter                   0.878729
clothing                  0.989242
money                     0.988753
missing_people            0.991687
refugees                  0.979951
death                     0.971149
other_aid                 0.859658
infrastructure_related    0.975550
transport                 0.981418
buildings                 0.958924
electricity               0.995110
tools                     0.997555
hospitals                 0.994132
shops                     0.997555
aid_centers               0.995599
other_infrastructure      0.986308
weather_r

In [177]:
y_pred = model.predict(X_test)
y_pred = pd.DataFrame(y_pred, columns=category_names)
y_pred

,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,child_alone,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2040,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2041,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2042,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2043,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [185]:
accuracy = (y_pred == Y_test).mean()

In [183]:
Y_test = Y_test.reset_index(drop=True)

In [203]:
Y_test

,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,child_alone,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
591,1.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3094,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6048,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4616,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0
3605,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7076,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0
7125,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
560,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1024,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
